In [2]:
# pancil method

using LinearAlgebra
module FastPolynomialRoots
using LibAMVW_jll, Polynomials
Polynomials.roots(p::Union{Polynomial{Float64},Polynomial{Complex{Float64}}}) = rootsFastPolynomialRoots(coeffs(p))
Polynomials.roots(p::Polynomial{T}) where {T <:Integer} = roots(convert(Polynomial{float(T)}, p))
function rootsFastPolynomialRoots(a::Vector{Float64})
    pl    = reverse!(a[1:end - 1] ./ a[end])
    np    = length(pl)
    reigs = similar(pl)
    ieigs = similar(pl)
    its   = Vector{Int32}(undef, np)
    flag  = Int32[0]

    ccall((:damvw_, libamvwdouble), Cvoid,
        (Ref{Int32}, Ptr{Float64}, Ptr{Float64}, Ptr{Float64}, Ptr{Int32}, Ptr{Int32}),
        np, pl, reigs, ieigs, its, flag)

    if flag[1] != 0
        error("error code: $(flag[1])")
    end
    return complex.(reigs, ieigs)
end

function rootsFastPolynomialRoots(a::Vector{Complex{Float64}})
    pl    = reverse!(a[1:end - 1] ./ a[end])
    plr   = real(pl)
    pli   = imag(pl)
    np    = length(pl)
    reigs = similar(plr)
    ieigs = similar(plr)
    its   = Vector{Int32}(undef, np)
    flag  = Int32[0]

    ccall((:zamvw_, libamvwsingle), Cvoid,
        (Ref{Int32}, Ptr{Float64}, Ptr{Float64}, Ptr{Float64}, Ptr{Float64}, Ptr{Int32}, Ptr{Int32}),
        np, plr, pli, reigs, ieigs, its, flag)

    if flag[1] != 0
        error("error code: $(flag[1])")
    end
    return complex.(reigs, ieigs)
end

end # module

Main.FastPolynomialRoots

In [3]:
using NPZ
using Test, Polynomials, LinearAlgebra
Qp_r = (npzread("Qp_r.npy"))
Qp_i = (npzread("Qp_i.npy"))
println(npzread("vs_r.npy")[1:20])
println(npzread("vs_i.npy")[1:20])

[0.01945870859284519, 0.011623035097551056, 0.006260690553425728, 0.004321396030051534, 0.0029427789696667636, 0.002587127165773514, 0.002294380625496655, 0.002043756268045034, 0.0018799536545543625, 0.001057410897285507, 0.0009836140050846016, 0.0009049440188799685, 0.0006552013868629646, 0.00027393692604621477, 0.00011440636516147428, 4.632631180419385e-5, 1.1733147914020653e-5, 2.9644619570952803e-6, 9.943698176301225e-7, 3.5904545562697633e-7]
[0.07725413357497686, 0.04756306111063494, 0.02352386538424301, 0.011958997091005794, 0.003129659492748492, 0.002898355155458097, 0.0025582724321977527, 0.0009350014061693463, 0.0005498214751823151, 0.0005217883863736603, 0.00031548738709639615, 0.0002424237012540267, 0.00015417077485146519, 0.00011863425644648085, 0.0001014651504778381, 6.875101368965793e-5, 4.646495187115566e-5, 1.3381407039872388e-5, 1.4889731272329147e-6, 6.949922618163177e-7]


In [5]:
i = 2
println(sum(real(Qp_r[:, i]).^2))
println(sum(imag(Qp_r[:, i]).^2))

if sum(real(Qp_r[:, i]).^2) < 1e-15
    gamma_r = roots(Polynomial(imag(Qp_r)[:, i]))
    println("imag")
else
    gamma_r = roots(Polynomial(real(Qp_r)[:, i]))
    println("real")
end
npzwrite("gamma_new_r.npy", gamma_r)


println(sum(real(Qp_i[:, i]).^2))
println(sum(imag(Qp_i[:, i]).^2))
if sum(real(Qp_i[:, i]).^2) < 1e-15
    gamma_i = roots(Polynomial(imag(Qp_i)[:, i]))
    println("imag")
else
    gamma_i = roots(Polynomial(real(Qp_i)[:, i]))
    println("real")
end
npzwrite("gamma_new_i.npy", gamma_i)

3.749399456654646e-33
1.0
imag
3.7493994566546427e-33
1.0
 Random shift!
imag


In [ ]:
length(Qp_r[:, 1])